In [1]:
# configure keys
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
MOSAIC_API_KEY = os.environ["MOSAIC_API_KEY"]


## Set up OpenAI and Mosaic Gateway Routes
See [config.yaml](./config.yaml) for the configuration. It looks like this:

```yaml
routes: 
  - name: chat_openai
    route_type: llm/v1/chat
    model:
      provider: openai
      name: gpt-3.5-turbo
      config:
        openai_api_key: $OPENAI_API_KEY
  - name: chat_mosaic_llama
    route_type: llm/v1/chat
    model:
      provider: mosaic
      name: llama2-70b-chat
      config:
        mosaic_api_key: $MOSAIC_API_KEY
```

In [2]:
import mlflow
from mlflow.gateway import MlflowGatewayClient


/Users/daniel.liden/projects/prompt_ui/env/lib/python3.10/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/daniel.liden/projects/prompt_ui/env/lib/python3.10/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [ ]:
#!mlflow gateway start --config-path config.yaml --port 5000 --host localhost --workers 2

In [3]:
# let's run that as a subprocess
import subprocess

cmd = ["mlflow", "gateway", "start", "--config-path", "config.yaml", "--port", "5000", "--host", "localhost", "--workers", "2"]
process = subprocess.Popen(cmd)

/Users/daniel.liden/projects/prompt_ui/env/lib/python3.10/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/daniel.liden/projects/prompt_ui/env/lib/python3.10/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
[2023-09-15 12:12:11 -0400] [16893] [INFO] Starting gunicorn 21.2.0
[2023-09-15 12:12:11 -0400] [16893] [INFO] Listening at: http://127.0.0.1:5000 (16893)
[2023-09-15 12:12:11 -0400] [16893] [INFO] Using worker: uvicorn.workers.UvicornWorker
[2023-09-15 12:12:11 -0400] [16894] [INFO] Booting worker with pid: 16894
[2023-09-15 12:12:11 -0400] [16895] [INFO] Booting worker with pid: 16895
/Users/daniel.liden/projects/prompt_ui/env/lib/pyt

In [17]:
# uncomment and run if you need to terminate the process
#process.terminate()

In [4]:

gateway_client = MlflowGatewayClient("http://localhost:5000")

In [5]:
gateway_client.search_routes()

[Route(name='chat_openai', route_type='llm/v1/chat', model=RouteModelInfo(name='gpt-3.5-turbo', provider='openai'), route_url='http://localhost:5000/gateway/chat_openai/invocations'),
 Route(name='chat_mosaic_llama', route_type='llm/v1/chat', model=RouteModelInfo(name='llama2-70b-chat', provider='mosaicml'), route_url='http://localhost:5000/gateway/chat_mosaic_llama/invocations')]

## Run a Test Query against each route

In [6]:
# test the openai route
response = gateway_client.query(
    "chat_openai", {"messages": [{"role": "user", "content": "Tell me a joke about rabbits"}]}
)
print(response)

{'candidates': [{'message': {'role': 'assistant', 'content': 'Why did the rabbit go to the barber?\n\nBecause it wanted a hare-cut!'}, 'metadata': {'finish_reason': 'stop'}}], 'metadata': {'input_tokens': 13, 'output_tokens': 16, 'total_tokens': 29, 'model': 'gpt-3.5-turbo-0613', 'route_type': 'llm/v1/chat'}}


In [8]:
response = gateway_client.query(
    "chat_mosaic_llama", {"messages": [{"role": "user", "content": "Tell me a joke about rabbits"}]}
)
print(response)

{'candidates': [{'message': {'role': 'assistant', 'content': ' Sure, here\'s a classic one:\n\nWhy did the rabbit go to the doctor?\n\nBecause he had hare-loss!\n\n(This joke plays on the word "hare," which is a synonym for rabbit, and "hair loss," which is a common medical condition in humans.)\n\nI hope that made you hop with laughter!'}, 'metadata': {'finish_reason': None}}], 'metadata': {'input_tokens': None, 'output_tokens': None, 'total_tokens': None, 'model': 'llama2-70b-chat', 'route_type': 'llm/v1/chat'}}


## Use in MLflow Prompt Engineering UI

In your terminal, run the following to tell MLflow where to find the gateway routes.

`export MLFLOW_GATEWAY_URI="http://localhost:5000"`

In [15]:
# create a new mlflow experiment
mlflow.set_experiment("mlflow_ui_example")

2023/09/15 12:28:22 INFO mlflow.tracking.fluent: Experiment with name 'mlflow_ui_example' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///Users/daniel.liden/projects/prompt_ui/mlruns/749607414286048058', creation_time=1694795302669, experiment_id='749607414286048058', last_update_time=1694795302669, lifecycle_stage='active', name='mlflow_ui_example', tags={}>

Start the mlflow ui with:
`mlflow ui --port 5001`

(It is only necessary to specify the port if you have something else running on port 5000; in this case, I started the AI gateway on port 5000).

We will do everything else in the UI!

## Start a new run with the prompt UI:

![](./screenshots/1.png)

## Select a model and configure a prompt template

![](./screenshots/2.png)

## Add and compare more models and more prompt inputs

![](./screenshots/3.png)

![](./screenshots/4.png)